In [24]:
import pandas as pd
import optuna
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from mlflow.tracking import MlflowClient

In [25]:

df = pd.read_csv("../dataset/output/completed_dataset.csv")
df["Date_Sold"] = pd.to_datetime(df["Date_Sold"])
df["Year_Sold"] = df["Date_Sold"].dt.year
df["Month_Sold"] = df["Date_Sold"].dt.month
df = df.drop(columns= ['Listing_ID','Agency_Name', 'Postcode', 'Date_Sold', 'Address', 'Suburb', 'Longitude', 'Latitude', 'Primary_School_Name',
       'Secondary_School_Name'] )


df.head(2)

,Price,Property_Type,Bedrooms,Bathrooms,Parking_Spaces,Land_Size,Primary_School_Distance,Secondary_School_Distance,Distance_to_CBD,Distance_to_Coast,Secondary_ICSEA,Primary_ICSEA,Year_Sold,Month_Sold
0,880000,1,3,2,5,1533,1149,508,17721,26634.585679,1067,1043,2023,8
1,3850000,1,5,3,3,1532,418,418,9983,1539.284806,1197,1197,2016,6


In [26]:
def catag_feature():
    """
    category features into numeric and category features for data processing in pipeline
    """
    numeric_features = ['Property_Type', 'Bedrooms', 'Bathrooms', 'Parking_Spaces', 'Land_Size', 'Primary_School_Distance', 
                        'Secondary_School_Distance', 'Distance_to_CBD', 'Distance_to_Coast', 'Secondary_ICSEA', 
                        'Primary_ICSEA', 'Year_Sold', 'Month_Sold']
    
    category_features = []
    
    return category_features, numeric_features

In [27]:
# define Features (X) and target (y)
X = df.drop(columns=['Price'])  
y = df['Price']   

# split training and testing dataset
X_train_full, X_val, y_train_full, y_val = train_test_split(X, y, test_size=0.25, random_state=100)





# Enable automatic logging for XGBoost
mlflow.xgboost.autolog()

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "objective": "reg:squarederror"
    }

    with mlflow.start_run(nested=True):
        category_features, numeric_features = catag_feature()
        
        numeric_transformer = Pipeline(
            steps=[
                ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())
                ]
        )
        # categorical_transformer = Pipeline(
        #     steps=[
        #         ("encoder", OneHotEncoder(handle_unknown="ignore"))
        #     ]
        # )

        preprocessor = ColumnTransformer(
            transformers=[
                ("numerical", numeric_transformer, numeric_features),
                # ("cat", categorical_transformer, category_features),
            ]
        )

        pipeline = Pipeline([
            ("preprocessing", preprocessor),
            ("model", xgb.XGBRegressor(**params))
        ])
        
        # Fit pipeline
        pipeline.fit(X_train_full, y_train_full)

        # loging pipeline
        mlflow.sklearn.log_model(pipeline, "model")

        # Predict and evaluate
        preds = pipeline.predict(X_val)
        rmse = root_mean_squared_error(y_val, preds)

        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("rmse", rmse)

        return rmse

# Start the main MLflow experiment
mlflow.set_tracking_uri(uri="http://localhost:8080/")
mlflow.set_experiment("housing-price-prediction")


with mlflow.start_run(run_name="optuna_hpo"):
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50)

    best_params = study.best_trial.params
    best_rmse = study.best_value

    mlflow.log_params(best_params)
    mlflow.log_metric("best_rmse", best_rmse)

2025/08/03 18:31:43 INFO mlflow.tracking.fluent: Experiment with name 'housing-price-prediction' does not exist. Creating a new experiment.
[I 2025-08-03 18:31:44,403] A new study created in memory with name: no-name-75a81f72-bd4f-4abb-9a39-3cb0b4fc9840
2025/08/03 18:31:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:31:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/MLOpsZoomcamp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [18:31:46] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1730232572796/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/08/03 18:31:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:31:56 WARNING mlflow.models.model: Model logged without a signature and 

🏃 View run unique-chimp-831 at: http://localhost:8080/#/experiments/1/runs/aaec0205c608420288657d813a1f37bc
🧪 View experiment at: http://localhost:8080/#/experiments/1


2025/08/03 18:31:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:31:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/MLOpsZoomcamp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [18:31:58] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1730232572796/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/08/03 18:32:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:32:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-03 18:32:06,365] Trial 1 finished with value: 228264.69896727204 and parameters: {'n_estimators': 298, 'max_depth': 6, 'learning_rate

🏃 View run rebellious-fly-930 at: http://localhost:8080/#/experiments/1/runs/b4697581ca594f84a6c711f54a151e71
🧪 View experiment at: http://localhost:8080/#/experiments/1


2025/08/03 18:32:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:32:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/MLOpsZoomcamp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [18:32:10] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1730232572796/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/08/03 18:32:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:32:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-03 18:32:19,958] Trial 2 finished with value: 217754.90443137087 and parameters: {'n_estimators': 351, 'max_depth': 10, 'learning_rat

...

🏃 View run upbeat-smelt-949 at: http://localhost:8080/#/experiments/1/runs/abad27da010649d4b4a4c3dc57c1735a
🧪 View experiment at: http://localhost:8080/#/experiments/1


2025/08/03 18:40:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:40:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/MLOpsZoomcamp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [18:40:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1730232572796/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/08/03 18:40:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:40:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-03 18:40:37,996] Trial 48 finished with value: 210566.6913058495 and parameters: {'n_estimators': 448, 'max_depth': 7, 'learning_rate

🏃 View run legendary-conch-790 at: http://localhost:8080/#/experiments/1/runs/359cdfecf2a74bafbc99e92596884317
🧪 View experiment at: http://localhost:8080/#/experiments/1


2025/08/03 18:40:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:40:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/MLOpsZoomcamp/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [18:40:40] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1730232572796/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/08/03 18:40:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 18:40:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-03 18:40:48,394] Trial 49 finished with value: 210308.4480670425 and parameters: {'n_estimators': 447, 'max_depth': 7, 'learning_rate

🏃 View run receptive-ape-416 at: http://localhost:8080/#/experiments/1/runs/d99d896c9f484976b4b6d56b3543c152
🧪 View experiment at: http://localhost:8080/#/experiments/1
🏃 View run optuna_hpo at: http://localhost:8080/#/experiments/1/runs/141e8bd8174e42e1a81e43ab3be9ba56
🧪 View experiment at: http://localhost:8080/#/experiments/1


In [30]:
def model_regisry():

    from mlflow.entities import ViewType
    from datetime import datetime


    date = datetime.now().date # today date

    client = MlflowClient(tracking_uri= "http://localhost:8080/")

    runs = client.search_runs(
        experiment_ids='1',
        filter_string="metrics.rmse < 230000",
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=5,
        order_by=["metrics.rmse ASC"]
    )

    for run in runs:
        print(f"runID: {run.info.run_id}, name: {run.info.run_name}, rmse: {run.data.metrics["rmse"]}")

    # register the first model in the list
    print(f"Fisrt model in the list: {runs[0].info.run_id}")
    run_id = runs[0].info.run_id
    modelURL = f"runs:/{run_id}/model"

    # tags for register model
    tag = {
        "model_type": "xgbRegression",
        "owner": "wing",
        "data_version": "v1.0"
    }

    mlflow.register_model(model_uri=modelURL, 
                          name= "Perth housing price prediction",
                          tags= tag)
    

    registered_models = client.search_registered_models()

    for model in registered_models:
        model_name = model.name
        if model_name == "Perth housing price prediction":
            for v in model.latest_versions:
                model_version = v.version

            client.transition_model_version_stage(
                name= model_name,
                version= model_version,
                stage="Production",
            )
            print(f"Perth housing price prediction - {model_version} : moved to production")
        else: 
            pass

    
    return run_id, model_version, model_name


In [37]:
def export_model(model_name, experiement_id):
    client = MlflowClient()
    model_version = client.get_model_version(model_name, experiement_id)

    model_folder_name = model_version.source.replace("models:/", "")

    import shutil

    # moving model folder to depolyment
    source = f"../docker/mlflow-prefect/mlflow/mlartifacts/1/models/{model_folder_name}"
    destination = "../deploy/model/"

    shutil.copytree(src=source, dst=destination, dirs_exist_ok=True)



In [38]:
export_model("Perth housing price prediction", "1")

In [31]:
model_regisry()

runID: ed220bc52d964f369fe0b8ed9bf39ad8, name: adorable-tern-776, rmse: 209455.421559702
runID: 3010df279f934cb584a15c4ffa0d1295, name: luminous-dove-820, rmse: 209738.88294286685
runID: 80bae48c76794d26b822afc9878f423b, name: intrigued-swan-148, rmse: 210017.80703459049
runID: 6497780b38604731852d1fe82c9c5c0f, name: rogue-bug-996, rmse: 210035.84029857063
runID: d99d896c9f484976b4b6d56b3543c152, name: receptive-ape-416, rmse: 210308.4480670425
Fisrt model in the list: ed220bc52d964f369fe0b8ed9bf39ad8


Successfully registered model 'Perth housing price prediction'.
2025/08/03 18:42:04 WARNING mlflow.tracking._model_registry.fluent: Run with id ed220bc52d964f369fe0b8ed9bf39ad8 has no artifacts at artifact path 'model', registering model based on models:/m-f70ce8e745ad4ef0a4a37542b79a6cf5 instead
2025/08/03 18:42:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Perth housing price prediction, version 1
Created version '1' of model 'Perth housing price prediction'.
/var/folders/pz/dfbhr6012hnd3dxtrz8tyxnm0000gn/T/ipykernel_14205/3681877867.py:47: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_

Perth housing price prediction - 1 : moved to production


('ed220bc52d964f369fe0b8ed9bf39ad8', '1', 'Perth housing price prediction')

In [32]:
client = MlflowClient()
model_version = client.get_model_version("Perth housing price prediction", "1")

print("Source path:", model_version.source) 

Source path: models:/m-f70ce8e745ad4ef0a4a37542b79a6cf5


In [17]:
model_version.source.replace("models:/", "")

'm-f2b3160bebf74881aac843e6d3e2d679'